# Dockerfile

如果将改变了一些配置的Container在生成一个镜像？

使用Dockerfile!!

Dockerfile其实并不是向镜像里直接写入的，因为镜像是只读的。docker在这个时候创建了一个临时的容器，然后写入内容之后，再把临时容器删除。

## DockerFile使用说明

### 基本指令说明

#### ARG指令**
定义创建镜像过程中使用的变量，相当于我们为`docker build - -build-arg`赋值。

镜像编译结束后，这个变量将不会被保存

```
ARG version=1.0
```

#### FROM指令
指定我们要在哪个image之上再进行构建，尽量使用官方image进行base image

并且一个Dockerfile，必须要以From指令作为开头（ARG是唯一一个可以先于From命令的)

```
FROM debian:latest
```
#### LABLE指令
像是代码里的注释一样，一些概括的维护者信息。
```
LABLE author=harry
```

#### ENV指令
定义变量，可以在dockerfile下方进行使用

例如我们定义了 `ENV USER harry`

```
ENV FILE_LOCATION /usr/local/file
```

####  USER指令
指定运行容器时的用户是谁

#### WORKDIR指令
进入到我们指定的目录中，如果没有这个目录会自动进行创建，用WORKDIR，代替 RUN cd。尽量使用绝对目录，不要使用相对目录。

```
WORKDIR /usr/local
WORKDIR tomcat/config
# 可以连续指定路径，如果像上述一样，指定的路径为/usr/local/tomcat/configs
```

#### RUN指令
每执行一次RUN就是就会产生镜像的一层，使用 "&&" 将多个命令串联起来，如果需要换行在最后需要使用" " 反斜杠。环境的运行与搭建，大多数情况下需要这个命令

```
RUN yum update \
        && yum install -y nginx
#上述操作先更新yum，然后下载nginx
```

#### CMD指令
设置启动后默认执行的命令和参数。如果docker run 进行了指定了命令，例如 docker run -it … /bin/bash。则不会运行CMD中的命令

而且如果CMD定义多个，后面会覆盖之前的。

```
## 启动tomcat命令
CMD ['catalina.sh', 'run']
```

#### ENTRYPOINT指令
设置容器启动时默认执行的命令和参数，该命令会在启动容器后作为根命令执行

容器以应用程序或者服务执行, 并且ENTRYPOINT一定会执行

```
# 将一个shell脚本作为docker启动的入口。
ENTRYPOINT ['/entry.sh']
```

#### COPY指令
把本地文件拷贝到docker里，COPY指令优于ADD指令，如果需要添加远程文件可以使用 curl或者wget
```
COPY . /temp
```
#### ADD指令
是把本地的文件复制到docker里去，不过不光如此，还会对压缩文件自动进行解压缩
```
ADD . /temp
```
#### VOLUME指令
启动容器时，可以在本地或者是其他容器创建数据卷挂载点，用于存放数据库和持久化数据
```
#指定挂载点为 /temp/mount
VOLUME /temp/mount
```
#### EXPOSE指令
声明镜像内部服务监听的端口，一次可以暴露多个端口
```
#暴露22端口，和8888端口
EXPOSE 22 8888
```


### 区别
#### ARG和ENV的区别
- ARG时创建镜像过程中使用的变量，在启动后的容器中不能使用
- ENV在容器中可以使用
#### RUN，CMD以及ENTRYPOINT的相同点以及区别
RUN 和 CMD和ENTRYPOINT，都可以使用exec格式和shell格式

exec格式举例：
```
CMD ['/bin/echo', 'hello world']
```
shell格式举例：
```
CMD echo 'hello world'
```

RUN命令用于构建镜像，CMD和ENTRYPOINT用于指定容器启动时的一些默认指令和参数。

**CMD与ENTRYPOINT的共同点**

两者在dockerfile中各自都只能声明一次，声明多次，不会报错，但是只有最后一条命令会生效。

**CMD与ENTRYPOINT的区别**

- ENTRYPOINT是docker容器的主命令
- 默认的一些参数会在CMD中进行指定
- CMD会被作为命令或者参数在ENTRYPOINT 参数后追加。
- CMD可被覆盖，ENTRYPOINT不会被覆盖

```
ENTRYPOINT ['/bin/echo', 'hello']
CMD ['world']
```
如果我们运行

`ocker run -it < image >`会输出 hello world

而如果我们运行

`docker run -it < image > harry` 会输出 hello harry

这就是因为我上面说过的，如果run的时候没有指定CMD会执行，如果指定了命令就不会执行CMD了

